In [0]:
#Import packages
from keras import backend as K
import numpy as np
from random import randint, random,sample
from numpy import array
from numpy import argmax
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import  roc_auc_score, auc,r2_score,precision_score
from matplotlib import pyplot as plt
import math
from numpy import unique
import pickle
#from Bio import Seq, SeqIO
#from Bio.Alphabet import generic_dna
#import dask.dataframe as dd
#from dask.multiprocessing import get
#from Bio.Alphabet.IUPAC import ambiguous_dna
from sklearn.metrics import r2_score, accuracy_score
from itertools import product
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from numpy.random import randint
from __future__ import print_function
from keras.models import Sequential, Model
from keras.layers import Concatenate, LeakyReLU, concatenate,GRU, Bidirectional, MaxPool1D,GlobalMaxPool1D,add
from keras.layers import Dense, Embedding, Input, Masking, Dropout, MaxPooling1D,Lambda, BatchNormalization, Reshape,UpSampling1D, AveragePooling1D
from keras.layers import LSTM, TimeDistributed, AveragePooling1D, Flatten,Activation,ZeroPadding1D, SeparableConv1D
from keras.optimizers import Adam, rmsprop
from keras.utils.training_utils import multi_gpu_model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping,ModelCheckpoint, CSVLogger
from keras.layers import Conv1D, GlobalMaxPooling1D, ConvLSTM2D, Bidirectional,RepeatVector
from keras.regularizers import *
from keras import regularizers
from keras.layers import concatenate as concatLayer
from sklearn.metrics import matthews_corrcoef as MCC
from keras.utils import plot_model

Using TensorFlow backend.


### Reading SILVA & HVR

In [0]:
#Reading V2 High  Variable Region (HVR)
#For full training download this 
# https://www.arb-silva.de/fileadmin/silva_databases/release_132/Exports/SILVA_132_SSURef_Nr99_tax_silva.fasta.gz
path = 'PCR_SILVA_all/V2_SILVA_2.fa'

In [0]:
### Function for Nucleotide sequence one hot encoding
#1 Declare the alphabet
alphabet = 'ACGTNRYSWKMBDHV'
integer = [1,2,3,4,0]
#2 Declare mapping functions
char_to_int = {'A':1,'C':2,'G':3,'T':4,'R':5,'Y':6,'S':7,
               'W':8,'K':9,'M':10,'B':11,'D':12,'H':13,'V':14,'N':15}
int_to_char = {1:'A',2:'C',3:'G',4:'T',5:'R',6:'Y',7:'S',
               8:'W',9:'K',10:'M',11:'B',12:'D',13:'H',14:'V',15:'N'}
#3 convert char to number
def encode_nu(sequence):
    return array([char_to_int[char] for char in sequence])

# Decode a encoded string
def decode_nu(encoded):
    decoded = [int_to_char[integ] for integ in encoded]
    return decoded

In [0]:
def zero_pad(list1):
    return np.pad(list1,(0,max_len-len(list1)),'constant',constant_values=0).astype('uint8')

In [0]:
def encode_pad(seq):
    return zero_pad(encode_nu(str(record.seq)),maxlen=max_len)

### Next cell can be escaped by reading the SILVA_header.csv

In [0]:
#Reading the whole SILVA database header as a panda dataframe to import the complete taxonomy
"""
reads=[]
for record in SeqIO.parse('SILVA_132_SSURef_tax_silva.fasta', "fasta"):
    str_ = str(record.description).split(" ", 1)[1]
    id_ = str(record.name).split(' ')[0]
    if str_.count(';') == 6:
        [kingdom, phylum, class_,order,family,genus,species] = str_.split(';')
    elif str_.count(';') > 6:    
        list =  str_.split(';')
        [kingdom,phylum, class_,order,family,genus,species] = list[-7:]
    else:
        list =  str_.split(';')
        num = len(list)
        [kingdom,phylum, class_,order,family,genus,species][:num] = list
        [kingdom,phylum, class_,order,family,genus,species][num:] = '_'*(7-num)
    reads.append([id_,kingdom ,phylum, class_,order,family,genus,species])    
    
SILVA_header =pd.DataFrame(reads,columns=['id','kingdom','phylum','class_','order','family','genus','species'])
"""

'\nreads=[]\nfor record in SeqIO.parse(\'SILVA_132_SSURef_tax_silva.fasta\', "fasta"):\n    str_ = str(record.description).split(" ", 1)[1]\n    id_ = str(record.name).split(\' \')[0]\n    if str_.count(\';\') == 6:\n        [kingdom, phylum, class_,order,family,genus,species] = str_.split(\';\')\n    elif str_.count(\';\') > 6:    \n        list =  str_.split(\';\')\n        [kingdom,phylum, class_,order,family,genus,species] = list[-7:]\n    else:\n        list =  str_.split(\';\')\n        num = len(list)\n        [kingdom,phylum, class_,order,family,genus,species][:num] = list\n        [kingdom,phylum, class_,order,family,genus,species][num:] = \'_\'*(7-num)\n    reads.append([id_,kingdom ,phylum, class_,order,family,genus,species])    \n    \nSILVA_header =pd.DataFrame(reads,columns=[\'id\',\'kingdom\',\'phylum\',\'class_\',\'order\',\'family\',\'genus\',\'species\'])\n'

In [0]:
#SILVA_header.to_csv('SILVA_header_ALL_TAXA.csv')

In [0]:
SILVA_header = pd.read_csv('SILVA_header_ALL_TAXA.csv',index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
SILVA_header.head()

,id,Description,Domain,phylum,class_,order,family,genus,species,semicolon_count
0,GY187501.2.1421,Bacteria;Epsilonbacteraeota;Campylobacteria;Ca...,Bacteria,Epsilonbacteraeota,Campylobacteria,Campylobacterales,Helicobacteraceae,Helicobacter,unidentified,6
1,GY194060.4884.6412,Bacteria;Firmicutes;Bacilli;Lactobacillales;St...,Bacteria,Firmicutes,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,unidentified,6
2,AC201869.46386.47908,Bacteria;Proteobacteria;Gammaproteobacteria;En...,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacteriales,Enterobacteriaceae,Candidatus Regiella,Candidatus Regiella insecticola,6
3,GY222292.1.1479,Bacteria;Actinobacteria;Actinobacteria;Micromo...,Bacteria,Actinobacteria,Actinobacteria,Micromonosporales,Micromonosporaceae,Salinispora,unidentified,6
4,GY230930.2147890.2149418,Bacteria;Firmicutes;Bacilli;Lactobacillales;St...,Bacteria,Firmicutes,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,unidentified,6


In [0]:
def from_phylum_to_genus(df):
    class__ = ''.join(#'K_'+str(df['kingdom'])+'__'+
                      'P_'+str(df['phylum'])+'__'+
                      'C_'+str(df['class_'])+'__'+
                      'O_'+str(df['order'])+'__'+
                      'F_'+str(df['family'])+'__'+
                      'G_'+str(df['genus']))
    return class__

In [0]:
def from_phylum_to_species(df):
    class__ = ''.join(#'K_'+str(df['kingdom'])+'__'+
                      'P_'+str(df['phylum'])+'__'+
                      'C_'+str(df['class_'])+'__'+
                      'O_'+str(df['order'])+'__'+
                      'F_'+str(df['family'])+'__'+
                      'G_'+str(df['genus'])+'__'+
                      'S_'+str(df['species']))
    return class__

In [0]:
#SILVA_header['genus-'] = SILVA_header.apply(lambda row: from_phylum_to_genus(row),axis=1)

In [0]:
#len(unique(SILVA_header['species'])),len(unique(SILVA_header['genus'])),len(unique(SILVA_header['family']))

In [0]:
SILVA_header.index = SILVA_header['id']

In [0]:
SILVA_header.head()

,id,kingdom,phylum,class_,order,family,genus,species
id,,,,,,,,
GY187501.2.1421,GY187501.2.1421,Bacteria,Epsilonbacteraeota,Campylobacteria,Campylobacterales,Helicobacteraceae,Helicobacter,unidentified
GY194060.4884.6412,GY194060.4884.6412,Bacteria,Firmicutes,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,unidentified
AC201869.46386.47908,AC201869.46386.47908,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacteriales,Enterobacteriaceae,Candidatus Regiella,Candidatus Regiella insecticola
GY222292.1.1479,GY222292.1.1479,Bacteria,Actinobacteria,Actinobacteria,Micromonosporales,Micromonosporaceae,Salinispora,unidentified
GY230930.2147890.2149418,GY230930.2147890.2149418,Bacteria,Firmicutes,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,unidentified


### Reading the HVR fasta file

In [0]:
#Reading the database as a panda dataframe
reads=[]
for record in SeqIO.parse(path, "fasta"):
    id_ = str(record.description).split(" ")[2].split(":")[0]
    encoded = record.seq 
    reads.append([id_,encoded,len(record.seq)])    
    
df = pd.DataFrame(reads,columns=['id','seq','len'])

In [0]:
"""
#Find the longest read in all HVR
fasta_files = np.sort(os.listdir(path='./')).tolist()
fasta_files = fasta_files[12:24]
reads=[]
for file in fasta_files:
    for record in SeqIO.parse(file, "fasta"):
        id_ = str(record.description).split(" ")[2].split(":")[0]
        reads.append([id_,len(record.seq),file])    
    
df = pd.DataFrame(reads,columns=['id','len','file'])
one_hvr = ['V2_SILVA.fasta','V3_SILVA.fa.fasta','V4_SILVA.fa.fasta',
           'V5_SILVA.fa.fasta','V7_SILVA.fa.fasta','V8_SILVA.fa.fasta']
df_2 = df[df['file'].isin(one_hvr)]
max(df_2['len'])
"""

'\n#Find the longest read in all HVR\nfasta_files = np.sort(os.listdir(path=\'./\')).tolist()\nfasta_files = fasta_files[12:24]\nreads=[]\nfor file in fasta_files:\n    for record in SeqIO.parse(file, "fasta"):\n        id_ = str(record.description).split(" ")[2].split(":")[0]\n        reads.append([id_,len(record.seq),file])    \n    \ndf = pd.DataFrame(reads,columns=[\'id\',\'len\',\'file\'])\none_hvr = [\'V2_SILVA.fasta\',\'V3_SILVA.fa.fasta\',\'V4_SILVA.fa.fasta\',\n           \'V5_SILVA.fa.fasta\',\'V7_SILVA.fa.fasta\',\'V8_SILVA.fa.fasta\']\ndf_2 = df[df[\'file\'].isin(one_hvr)]\nmax(df_2[\'len\'])\n'

In [0]:
#df['Domain']  = df['id'].map(SILVA_header['Domain'])
df['phylum']  = df['id'].map(SILVA_header['phylum'])
df['class_']  = df['id'].map(SILVA_header['class_'])
df['order']   = df['id'].map(SILVA_header['order'])
df['family']  = df['id'].map(SILVA_header['family'])
df['genus']   = df['id'].map(SILVA_header['genus'])
df['species'] = df['id'].map(SILVA_header['species'])

In [0]:
def encode_label_to_close_int(df,genus_path,species_path,encode_species = True):   
    """
    input:
    df: dataframe of SILVA_taxa that
    genus_path :  path to save the pkl file of teh genus mapping
    species_path :  path to save the pkl file of teh species mapping

    
    output:
    df: the same dataframe with additional columns that represent one hot encoding 
                  that takes into account the hierarchical structure
    
    """
    """
    kingdoms_ = df['kingdom'].value_counts().index
    dict_ = dict()
    dict_ = dict_.fromkeys(kingdoms_)
    keys = range(len(kingdoms_))
    for k in keys:
        dict_[kingdoms_[k]] = keys[k]
    #df['kingdom-'] = df['kingdom'].apply(lambda x: dict_[x])
    df['Domain-'] = pd.factorize(df['Domain'].values)[0]
    df = df.sort_values(by='Domain-')"""
    df['phylum-'] = pd.factorize(df['phylum'].values)[0]
    df = df.sort_values(by=['phylum-'])
    df['class_-'] = pd.factorize(df['class_'].values)[0]
    df.sort_values(by=['class_-'])
    df['order-'] = pd.factorize(df['order'].values)[0]
    df.sort_values(by=['order-'])
    df['family-'] = pd.factorize(df['family'].values)[0]
    df.sort_values(by=['family-'])    
    #df = df[df['genus']!='uncultured']

    ## As some genus isn "unidetified" across many families, so 
    ## concatenate SILVA labels to one label from kingdom to genus
    ddata = dd.from_pandas(df, npartitions=30)
    df['Complete_genus'] = ddata.map_partitions(lambda df:df.apply(lambda row: 
                                                                   from_phylum_to_genus(row),
                                                                    axis=1)).compute(scheduler='threads')
    genus_mapping = pd.factorize(df['Complete_genus'].values)[1]
    df['genus-'] = pd.factorize(df['Complete_genus'].values)[0]
    df = df.sort_values(by=['genus-'])
    #save this mapping as a pickle file for all HVR models
    with open(genus_path, 'wb') as fp:
        pickle.dump(genus_mapping, fp)
    
    if encode_species == True:
        ddata = dd.from_pandas(df, npartitions=30)
        df['Complete_species'] = ddata.map_partitions(lambda df:df.apply(lambda row: 
                                                                                 from_phylum_to_species(row),
                                                                                 axis=1)).compute(scheduler='threads')
        species_mapping = pd.factorize(df['Complete_species'].values)[1]

        df['species-'] = pd.factorize(df['Complete_species'].values)[0]
        df = df.sort_values(by=['species-'])
        #save this mapping as a pickle file for all HVR models
        with open(species_path, 'wb') as fp:
            pickle.dump(species_mapping, fp)
    return df

In [0]:
df = encode_label_to_close_int(df,'./PCR_silva_all/data/V2_genus_mapping.pkl',
                               './PCR_silva_all/data/V2_species_mapping.pkl',encode_species = True)

In [0]:
df = df[df['phylum-']!=-1]

In [0]:
df.shape

(1089829, 19)

In [0]:
df = df.drop(columns=[ 'phylum', 'class_', 'order', 'family','genus', 'species'])

In [0]:
d = Seq.IUPAC.IUPACData.ambiguous_dna_values
ambiguous_ch = d.keys()- ['A','G','C','T']
ambiguous_ch
df['ambiguity_count'] = df['seq'].apply(lambda x: sum([''.join(x).count(y) for y in ambiguous_ch]))

In [0]:
df.to_pickle('PCR_SILVA_all/data/V2_df_without_encoding.pkl')

### Reading V2 dataframe after preprocessing

In [0]:
df = pd.read_pickle('PCR_SILVA_all/data/V2_df_without_encoding.pkl')

In [0]:
df_amb = df[df['ambiguity_count']>0]

In [0]:
def expend_ambiguity_df(df_amb):
    
    df_amb = df_amb[df_amb['ambiguity_count']<10]
    df_amb = df_amb.reset_index().drop(columns=['index'])
    new_df = pd.DataFrame(columns=df_amb.columns)
    new_df['seq-'] = ''
    for i in range(df_amb.shape[0]):
        sample =  df_amb.iloc[i,:]
        acceptable_moves = len(list(map("".join, product(*map(d.get, sample['seq'])))))
        seq = sample['seq']
        
        if acceptable_moves in [1,2,3]:
            new_sample = pd.DataFrame(columns=df_amb.columns)
            new_sample = new_sample.append([sample]*1)
            choice = randint(1,acceptable_moves,1)
            
            x = [x[y] for y in choice]
            new_sample['seq-'] = x
            new_df = pd.concat([new_df,new_sample])
        else:
            new_sample = pd.DataFrame(columns=df_amb.columns)
            new_sample = new_sample.append([sample]*1)
            choice = randint(1,acceptable_moves,1)
            x = list(map("".join, product(*map(d.get, seq))))
            x = [x[y] for y in choice]          
            new_sample['seq-'] = x
            new_df = pd.concat([new_df,new_sample])
    return new_df

In [0]:
df_amb = expend_ambiguity_df(df_amb)

In [0]:
df = df[df['ambiguity_count']==0]
df['seq-'] = df['seq']

In [0]:
df = pd.concat([df,df_amb])
df = df.drop(columns=['seq'])

In [0]:
# Applying encoding
df['encoded'] = df['seq-'].apply(encode_nu)

In [0]:
df = df.drop(columns=['seq-'])

In [0]:
df = df.drop(columns=[ 'len', 'Complete_genus'])

In [0]:
df.to_pickle('PCR_SILVA_all/data/V2_df_after_encoding_&_ambiguity_exp.pkl')

### Apply stratified sampling to have at least equal classes in Train, Test, Validation data



In [0]:
df = pd.read_pickle('PCR_SILVA_all/data/V2_df_after_encoding_&_ambiguity_exp.pkl')

In [0]:
df = df.sample(frac=1).reset_index(drop=True)

In [0]:
counts = df['species-'].value_counts()
df = df[df['species-'].isin(counts[counts > 2].index)]

In [0]:
train,test,_,_ = train_test_split(df,df['species-'],random_state= 1,test_size = 0.2)
train,valid,_,_ = train_test_split(train,train['species-'],random_state= 1,test_size = 0.15)

In [0]:
train.shape, test.shape, valid.shape

((624974, 10), (183817, 10), (110290, 10))

In [0]:
len(np.unique(train['species-'])),len(np.unique(test['species-'])),len(np.unique(valid['species-']))

(18725, 13918, 11145)

In [0]:
train = train[train['species-'].isin(unique(valid['species-']))]
test = test[test['species-'].isin(unique(valid['species-']))]

In [0]:
len(np.unique(train['species-'])),len(np.unique(test['species-'])),len(np.unique(valid['species-']))

(11019, 8563, 11145)

In [0]:
train = train[train['species-'].isin(unique(test['species-']))]
valid = valid[valid['species-'].isin(unique(test['species-']))]

In [0]:
len(np.unique(train['species-'])),len(np.unique(test['species-'])),len(np.unique(valid['species-']))

(8443, 8563, 8563)

In [0]:
test  = test[test['species-'].isin(unique(train['species-']))]
valid = valid[valid['species-'].isin(unique(train['species-']))]

In [0]:
len(np.unique(train['species-'])),len(np.unique(test['species-'])),len(np.unique(valid['species-']))

(8443, 8443, 8443)

In [0]:
# Reducing labels to number of integers in all train, test , valid
train['type'] = 'train'
test['type'] = 'test'
valid['type'] = 'valid'

df = pd.concat([train,test,valid])
df = df.sort_values(by=['phylum-'])
df['phylum-']= pd.factorize(df['phylum-'])[0]
df = df.sort_values(by=['class_-'])
df['class_-']= pd.factorize(df['class_-'])[0]
df = df.sort_values(by=['order-'])
df['order-']= pd.factorize(df['order-'])[0]
df = df.sort_values(by=['family-'])
df['family-']= pd.factorize(df['family-'])[0]
df = df.sort_values(by=['genus-'])
df['genus-']= pd.factorize(df['genus-'])[0]
species_mapping_2 = pd.factorize(df['Complete_species'])[1]
with open('PCR_SILVA_all/data/species_mapping_2.pkl', 'wb') as fp:
    pickle.dump(species_mapping_2, fp)
df['species-']= pd.factorize(df['Complete_species'])[0]

In [0]:
train = df[df['type']=='train'].drop(columns=['type'])
test =  df[df['type']=='test'].drop(columns=['type'])
valid = df[df['type']=='valid'].drop(columns=['type'])

In [0]:
valid.to_pickle('PCR_SILVA_all/data/V2_valid.pkl')
test.to_pickle('PCR_SILVA_all/data/V2_test.pkl')
train.to_pickle('PCR_SILVA_all/data/V2_train.pkl')

In [0]:
np.max(df['phylum-'])

24

In [0]:
np.max(df['class_-'])

47

In [0]:
np.max(df['order-'])

153

In [0]:
np.max(df['family-'])

325

In [0]:
np.max(df['genus-'])

1552

In [0]:
np.max(df['species-'])

8442

### Training Word2Vec on encoded sequences

In [0]:
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors

In [0]:
train = pd.read_pickle('PCR_SILVA_all/data/V2_train.pkl')
valid = pd.read_pickle('PCR_SILVA_all/data/V2_valid.pkl')

In [0]:
np.random.seed(1)
train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)

In [0]:
df = pd.concat([train,valid])

In [0]:
kmer_size = 6

In [0]:
import itertools
bases=['1','2','3','4']
all_kmers = [''.join(p) for p in itertools.product(bases, repeat=kmer_size)]
word_to_int = dict()
word_to_int = word_to_int.fromkeys(all_kmers)
keys = range(1,len(all_kmers)+1)
for k in keys:
    word_to_int[all_kmers[k-1]] = keys[k-1]

In [0]:
def oneHotEncoding_to_kmers(encoded_list,kmer_size):
    word_list = []
    ch_str = str(encoded_list.tolist()).replace(',','').replace('[','').replace(']','').replace(' ','')
    for i in range(len(ch_str) - kmer_size + 1):
        word_list.append(word_to_int[ch_str[ i : i + kmer_size ]])
    return word_list

In [0]:
def save_kmerized_corpus(path, kmer_size,train):
    with open(path, 'w') as file:
        for i in range(train.shape[0]):
            kmers = oneHotEncoding_to_kmers(train['encoded'].iloc[i],kmer_size)
            file.write(str(kmers).replace(',','').replace('[','').replace(']',''))
            file.write('\n')

In [0]:
from gensim.models.word2vec import Word2Vec,LineSentence
from gensim.models import KeyedVectors
import multiprocessing

In [0]:
#Saving the corpus to the disk to save Memory
save_kmerized_corpus('W2V_Kmerized_encoded_corpus.txt',kmer_size,df)

In [0]:
from gensim.models.word2vec import Word2Vec,LineSentence
from gensim.models import KeyedVectors
import multiprocessing

corpus = LineSentence(source='W2V_Kmerized_encoded_corpus.txt')

# Gensim Word2Vec model
vector_size = 150
window_size = 50
max_features = 4**kmer_size + 1
max_len = 321

# Create Word2Vec
word2vec = Word2Vec(sentences=corpus,
                    sg=1,
                    max_vocab_size=max_features,
                    sample=1e-5,
                    size=vector_size, 
                    window=window_size,
                    hs=1,
                    negative=20,
                    iter=5,
                    workers=multiprocessing.cpu_count(),
                    seed=5)
word2vec.wv.save_word2vec_format('PCR_SILVA_all/data/V2_word2vec_model_'+str(kmer_size)+'_kmer_'+'.w2v')

### Build HVR full length dataframe

In [0]:
valid = pd.read_pickle('PCR_SILVA_all/data/V2_valid.pkl')
train = pd.read_pickle('PCR_SILVA_all/data/V2_train.pkl')

In [0]:
train = train.sample(frac=1).reset_index(drop=True)
valid = valid.sample(frac=1).reset_index(drop=True)

In [0]:
train = train.drop(columns=['Complete_species','ambiguity_count'])
valid = valid.drop(columns=['Complete_species','ambiguity_count'])

In [0]:
kmer_size = 6
max_features = 4**kmer_size +1
vector_size = 128

In [0]:
def build_embedding_matrix(model_path,word_to_int):
    word2vec = KeyedVectors.load_word2vec_format(model_path)
    embedding_matrix = np.zeros((max_features,vector_size))
    for i in word_to_int.values():
        embedding_matrix[i,:] = word2vec.wv[str(i)]
    return embedding_matrix

In [0]:
def oneHotEncoding_to_kmers(encoded_list,kmer_size):
    word_list = []
    ch_str = str(encoded_list.tolist()).replace(',','').replace('[','').replace(']','').replace(' ','')
    for i in range(len(ch_str) - kmer_size + 1):
        word_list.append(int(word_to_int[ch_str[ i : i + kmer_size ]]))
    return word_list

In [0]:
import itertools
from gensim.models import KeyedVectors
bases=['1','2','3','4']
all_kmers = [''.join(p) for p in itertools.product(bases, repeat=kmer_size)]
word_to_int = dict()
word_to_int = word_to_int.fromkeys(all_kmers)
keys = range(1,len(all_kmers)+1)
for k in keys:
    word_to_int[all_kmers[k-1]] = keys[k-1]

In [0]:
embedding_matrix = build_embedding_matrix('PCR_SILVA_all/data/V2_word2vec_model_'+str(kmer_size)+'kmer.w2v',
                                          word_to_int)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


In [0]:
train['encoded'] = train['encoded'].apply(lambda x: oneHotEncoding_to_kmers(x,kmer_size=kmer_size)).values.tolist()
valid['encoded'] = valid['encoded'].apply(lambda x: oneHotEncoding_to_kmers(x,kmer_size=kmer_size)).values.tolist()

In [0]:
test  = pd.read_pickle('PCR_SILVA_all/data/V2_test.pkl')
test  = test.sample(frac=1).reset_index(drop=True)
test  = test.drop(columns=['Complete_species','ambiguity_count'])
test['encoded']  = test['encoded'].apply(lambda  x: oneHotEncoding_to_kmers(x,kmer_size=kmer_size)).values.tolist()

In [0]:
classes_1 = 24   +1
classes_2 = 47   +1
classes_3 = 153  +1
classes_4 = 325  +1
classes_5 = 1552 +1
classes_6 = 8442 +1 #10

max_len = 320 + 1

In [0]:
#A function to convert ambiguity encoding to zeros
ch = [1,2,3,4]
def zeros_for_ambguities(encode):
    return [y if y in ch else 0 for y in encode]
#valid['encoded'] = valid['encoded'].apply(zeros_for_ambguities)
#train['encoded'] = train['encoded'].apply(zeros_for_ambguities)
#test['encoded']  =  test['encoded'].apply(zeros_for_ambguities)

In [0]:
import random
def simulate_reads_range(list1,min_len=50):
    random.seed(random.randint(1,1000))
    start = randint(len(list1)-min_len)
    end = randint(start+min_len,len(list1))
    return list1[start:end]

In [0]:
def simulate_reads(list1,len1 = 100):
    start = randint(len(list1)-len1)
    return list1[start:start+len1]

In [0]:
def expand_by_class_count(df,min_samples = 10,duplication = 2):
    counts = df['genus-'].value_counts()
    very_rare_genus = df[df['genus-'].isin(counts[counts <=5].index)]
    rare_genus = df[df['genus-'].isin(counts[counts <=min_samples].index)]
    for i in range(duplication-1):
        df = pd.concat([df,rare_genus])
        for k in range(min_samples//2):
            df = pd.concat([df,very_rare_genus])
    return df

### Reading train, test , valid dataset

In [0]:
def simulate_multioutput_ngs_generator(df,len1 = 50,batch_size=250):
    y_sim_1,y_sim_2,y_sim_3,y_sim_4,y_sim_5,y_sim_6 = df['phylum-'].values,df['class_-'].values,df['order-'].values,df['family-'].values,df['genus-'].values,df['species-'].values
    x_sim = df['encoded'].apply(lambda x : simulate_reads_range(x,min_len=len1))
    x_sim = pad_sequences(x_sim.values,maxlen=max_len)
    x_sim = array(np.concatenate(x_sim).reshape(x_sim.shape[0],max_len).tolist()).astype('uint16')
    samples_per_epoch = df.shape[0]
    number_of_batches = samples_per_epoch//batch_size
    counter=0

    while counter <=number_of_batches :
        X_batch = np.array(x_sim[batch_size*counter:batch_size*(counter+1)]).astype('uint16')
        y_1 = y_sim_1[batch_size*counter:batch_size*(counter+1)].astype('uint8')
        y_2 = y_sim_2[batch_size*counter:batch_size*(counter+1)].astype('uint8')
        y_3 = y_sim_3[batch_size*counter:batch_size*(counter+1)].astype('uint8')
        y_4 = y_sim_4[batch_size*counter:batch_size*(counter+1)].astype('uint16')
        y_5 = y_sim_5[batch_size*counter:batch_size*(counter+1)].astype('uint16')
        y_6 = y_sim_6[batch_size*counter:batch_size*(counter+1)].astype('uint16')
        counter += 1
        if counter ==number_of_batches:
            counter = 0
            del x_sim,  y_sim_1,y_sim_2,y_sim_3,y_sim_4,y_sim_5,y_sim_6
            y_sim_1,y_sim_2,y_sim_3,y_sim_4,y_sim_5,y_sim_6 = df['phylum-'].values,df['class_-'].values,df['order-'].values,df['family-'].values,df['genus-'].values,df['species-'].values
            x_sim = df['encoded'].apply(lambda x : simulate_reads_range(x,min_len=len1))
            x_sim = pad_sequences(x_sim.values,maxlen=max_len)
            x_sim = array(np.concatenate(x_sim).reshape(x_sim.shape[0],max_len).tolist()).astype('uint16')
        yield X_batch, [y_1,y_2,y_3,y_4,y_5,y_6]

In [0]:
def simulate_ngs_generator(df,len1 = 50,batch_size=250):
    #y_sim_1,y_sim_2,y_sim_3,y_sim_4,y_sim_5,y_sim_6 = df['phylum-'].values,df['class_-'].values,df['order-'].values,df['family-'].values,df['genus-'].values,df['species-'].values
    y_sim_6 = df['species-'].values
    x_sim = df['encoded'].apply(lambda x : simulate_reads_range(x,min_len=len1))
    x_sim = pad_sequences(x_sim.values,maxlen=max_len)
    x_sim = array(np.concatenate(x_sim).reshape(x_sim.shape[0],max_len).tolist()).astype('uint16')
    samples_per_epoch = df.shape[0]
    number_of_batches = samples_per_epoch//batch_size
    counter=0

    while counter <=number_of_batches :
        X_batch = np.array(x_sim[batch_size*counter:batch_size*(counter+1)]).astype('uint16')
        #y_1 = y_sim_1[batch_size*counter:batch_size*(counter+1)].astype('uint8')
        #y_2 = y_sim_2[batch_size*counter:batch_size*(counter+1)].astype('uint8')
        #y_3 = y_sim_3[batch_size*counter:batch_size*(counter+1)].astype('uint8')
        #y_4 = y_sim_4[batch_size*counter:batch_size*(counter+1)].astype('uint16')
        #y_5 = y_sim_5[batch_size*counter:batch_size*(counter+1)].astype('uint16')
        y_6 = y_sim_6[batch_size*counter:batch_size*(counter+1)].astype('uint16')
        #y_6 = np.array(to_categorical(y_6,num_classes=classes_6)).astype('uint16')
        counter += 1
        if counter ==number_of_batches:
            counter = 0
            del x_sim,y_sim_6# y_sim_1,y_sim_2,y_sim_3,y_sim_4,y_sim_5,y_sim_6
            #y_sim_1,y_sim_2,y_sim_3,y_sim_4,y_sim_5,y_sim_6 = df['phylum-'].values,df['class_-'].values,df['order-'].values,df['family-'].values,df['genus-'].values,df['species-'].values
            y_sim_6 = df['species-'].values
            x_sim = df['encoded'].apply(lambda x : simulate_reads_range(x,min_len=len1))
            x_sim = pad_sequences(x_sim.values,maxlen=max_len)
            x_sim = array(np.concatenate(x_sim).reshape(x_sim.shape[0],max_len).tolist()).astype('uint8')
        yield X_batch, y_6#[y_1,y_2,y_3,y_4,y_5]

In [0]:
def simulate_ngs_generator_fixed_len(df,len1 = 100,batch_size=250):
    y_sim = df['species-'].values
    x_sim = df['encoded'].apply(lambda x : simulate_reads_range(x,min_len=len1))
    x_sim = pad_sequences(x_sim.values,maxlen=max_len)
    x_sim = array(np.concatenate(x_sim).reshape(x_sim.shape[0],max_len).tolist()).astype('uint8')
    samples_per_epoch = df.shape[0]
    number_of_batches = samples_per_epoch//batch_size
    counter=0

    while counter <=number_of_batches :
        X_batch = np.array(x_sim[batch_size*counter:batch_size*(counter+1)]).astype('uint8')
        y_batch = y_sim[batch_size*counter:batch_size*(counter+1)].astype('uint16')
        #y_batch = np.array(to_categorical(y_batch,num_classes=num_classes)).astype('uint16')
        counter += 1
        if counter ==number_of_batches:
            counter = 0
            del x_sim, y_sim
            y_sim = df['species-'].values
            x_sim = df['encoded'].apply(lambda x : simulate_reads_range(x,min_len=len1))
            x_sim = pad_sequences(x_sim.values,maxlen=max_len)
            x_sim = array(np.concatenate(x_sim).reshape(x_sim.shape[0],max_len).tolist()).astype('uint8')
        yield X_batch,y_batch

In [0]:
def simulate_ngs_df(df,len1 = 100):
    #y_sim = to_categorical(df['genus-'],num_classes=num_classes)
    y_sim = df['genus-'].values
    df['simulated'] = df['encoded'].apply(lambda x : simulate_reads(x,len1=len1))
    df['simulated'] = pad_sequences(df['simulated'].values,maxlen=max_len).tolist()
    x_sim = df['simulated']
    x_sim = array(np.concatenate(x_sim.values).reshape(x_sim.shape[0],max_len).tolist()).astype('uint8')
    return x_sim , y_sim

In [0]:
# AdamW
from keras.optimizers import Optimizer
from keras import backend as K
import six
import copy
from six.moves import zip
from keras.utils.generic_utils import serialize_keras_object
from keras.utils.generic_utils import deserialize_keras_object
from keras.legacy import interfaces

class AdamW(Optimizer):
    """Adam optimizer.
    Default parameters follow those provided in the original paper.
    # Arguments
        lr: float >= 0. Learning rate.
        beta_1: float, 0 < beta < 1. Generally close to 1.
        beta_2: float, 0 < beta < 1. Generally close to 1.
        epsilon: float >= 0. Fuzz factor.
        decay: float >= 0. Learning rate decay over each update.
        weight_decay: float >= 0. Decoupled weight decay over each update.
    # References
        - [Adam - A Method for Stochastic Optimization](http://arxiv.org/abs/1412.6980v8)
        - [Optimization for Deep Learning Highlights in 2017](http://ruder.io/deep-learning-optimization-2017/index.html)
        - [Fixing Weight Decay Regularization in Adam](https://arxiv.org/abs/1711.05101)
    """

    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4,  # decoupled weight decay (1/4)
                 epsilon=1e-8, decay=0., **kwargs):
        super(AdamW, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            self.wd = K.variable(weight_decay, name='weight_decay') # decoupled weight decay (2/4)
        self.epsilon = epsilon
        self.initial_decay = decay

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]
        wd = self.wd # decoupled weight decay (3/4)

        lr = self.lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * K.cast(self.iterations,
                                                  K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        self.weights = [self.iterations] + ms + vs

        for p, g, m, v in zip(params, grads, ms, vs):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon) - lr * wd * p # decoupled weight decay (4/4)

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'weight_decay': float(K.get_value(self.wd)),
                  'epsilon': self.epsilon}
        base_config = super(AdamW, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

### ResNet like network

In [0]:
# 46 epoch
#inp = Input(shape=(max_len,), dtype='uint8')
inp = Input(shape=(max_len,),dtype='uint16')
main = Embedding(max_features, vector_size, weights=[embedding_matrix],trainable=False)(inp)
#main = Embedding(5, 128)(inp)
#main = SpatialDropout1D(dr_rate)(main)
main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
i_l1 = MaxPooling1D(pool_size=2)(main)
main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=64, kernel_size=3, padding='same')(main)

main = concatenate([main, i_l1],axis=1)

main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)
i_l1 = Conv1D(filters=128, kernel_size=1, padding='same')(main)

main = Conv1D(filters=128, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=128, kernel_size=3, padding='same')(main)
main = concatenate([main, i_l1],axis=1)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)
i_l1 = Conv1D(filters=256, kernel_size=1, padding='same')(main)

main = Conv1D(filters=256, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=256, kernel_size=3, padding='same')(main)
main = concatenate([main, i_l1],axis=1)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)
i_l1 = Conv1D(filters=512, kernel_size=1, padding='same')(main)

main = Conv1D(filters=512, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=512, kernel_size=3, padding='same')(main)
main = concatenate([main, i_l1],axis=1)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)
main = GlobalMaxPool1D()(main)

main = Dense(1024, activation='relu')(main)  # Should be 4096
main = Dense(512, activation='relu')(main)  # Should be 2048

out1 = Dense(classes_1,activation='softmax')(main)
out2 = Dense(classes_2,activation='softmax')(main)
out3 = Dense(classes_3,activation='softmax')(main)
out4 = Dense(classes_4,activation='softmax')(main)
out5 = Dense(classes_5,activation='softmax')(main)
out6 = Dense(classes_6,activation='softmax')(main)

m = Model(inputs=[inp], outputs=[out1,out2,out3,out4,out5,out6])

optimizer = AdamW(lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4, epsilon=1e-8, decay=0.)
m.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=['acc'])

In [0]:
batch_size = 250
m.load_weights('./PCR_SILVA_all/W2V/ResNet_like_N_big.hdfs')
filepath = ''.join('./PCR_SILVA_all/W2V/ResNet_like_N_big_len50.hdfs')
Checkpoint = ModelCheckpoint(filepath,monitor='val_loss', save_best_only=True,
                             save_weights_only=True,period=1)
csv_logger = CSVLogger('./PCR_SILVA_all/W2V/ResNet_like_N_big_len50.log')
EarlyStop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

m.fit_generator(simulate_multioutput_ngs_generator(train,batch_size=batch_size,len1=50),
                epochs=50,shuffle=True,
                steps_per_epoch=train.shape[0]//batch_size,
                validation_data=simulate_multioutput_ngs_generator
                (valid,batch_size=batch_size,len1=50),
                validation_steps=valid.shape[0]//batch_size,
                verbose=2,callbacks=[Checkpoint,csv_logger,EarlyStop])

Epoch 1/50
 - 2070s - loss: 2.1622 - dense_3_loss: 0.0248 - dense_4_loss: 0.0416 - dense_5_loss: 0.1075 - dense_6_loss: 0.1679 - dense_7_loss: 0.4538 - dense_8_loss: 1.3667 - dense_3_acc: 0.9934 - dense_4_acc: 0.9887 - dense_5_acc: 0.9698 - dense_6_acc: 0.9558 - dense_7_acc: 0.8787 - dense_8_acc: 0.6808 - val_loss: 2.5957 - val_dense_3_loss: 0.0296 - val_dense_4_loss: 0.0469 - val_dense_5_loss: 0.1206 - val_dense_6_loss: 0.1887 - val_dense_7_loss: 0.5061 - val_dense_8_loss: 1.7038 - val_dense_3_acc: 0.9929 - val_dense_4_acc: 0.9883 - val_dense_5_acc: 0.9680 - val_dense_6_acc: 0.9530 - val_dense_7_acc: 0.8734 - val_dense_8_acc: 0.6574
Epoch 2/50
 - 1992s - loss: 2.1390 - dense_3_loss: 0.0245 - dense_4_loss: 0.0406 - dense_5_loss: 0.1061 - dense_6_loss: 0.1660 - dense_7_loss: 0.4491 - dense_8_loss: 1.3526 - dense_3_acc: 0.9936 - dense_4_acc: 0.9891 - dense_5_acc: 0.9702 - dense_6_acc: 0.9564 - dense_7_acc: 0.8801 - dense_8_acc: 0.6820 - val_loss: 2.5756 - val_dense_3_loss: 0.0289 - val_d

KeyboardInterrupt: ignored

### ResNet_Like Network_No_W2V

In [0]:
#inp = Input(shape=(max_len,), dtype='uint8')
inp = Input(shape=(max_len,),dtype='uint16')
#main = Embedding(max_features, vector_size, weights=[embedding_matrix],trainable=False)(inp)
main = Embedding(max_features, 128)(inp)
#main = SpatialDropout1D(dr_rate)(main)
main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
i_l1 = MaxPooling1D(pool_size=2)(main)

main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
main = concatenate([main, i_l1],axis=1)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)

i_l1 = Conv1D(filters=128, kernel_size=1, padding='same')(main)
main = Conv1D(filters=128, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=128, kernel_size=3, padding='same')(main)
main = concatenate([main, i_l1],axis=1)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)

i_l1 = Conv1D(filters=256, kernel_size=1, padding='same')(main)
main = Conv1D(filters=256, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=256, kernel_size=3, padding='same')(main)
main = concatenate([main, i_l1],axis=1)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)

i_l1 = Conv1D(filters=512, kernel_size=1, padding='same')(main)
main = Conv1D(filters=512, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=512, kernel_size=3, padding='same')(main)
main = concatenate([main, i_l1],axis=1)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)

main = GlobalMaxPool1D()(main)
main = Dense(1024, activation='relu')(main)  # Should be 4096
main = Dense(512, activation='relu')(main)  # Should be 2048

out1 = Dense(classes_1,activation='softmax')(main)
out2 = Dense(classes_2,activation='softmax')(main)
out3 = Dense(classes_3,activation='softmax')(main)
out4 = Dense(classes_4,activation='softmax')(main)
out5 = Dense(classes_5,activation='softmax')(main)
out6 = Dense(classes_6,activation='softmax')(main)

m = Model(inputs=[inp], outputs=[out1,out2,out3,out4,out5,out6])

optimizer = AdamW(lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4, epsilon=1e-8, decay=0.)
m.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=['acc'])

In [0]:
batch_size = 250
m.load_weights('./PCR_SILVA_all/W2V/ResNet_like_N_No_W2V.hdfs')
filepath = ''.join('./PCR_SILVA_all/W2V/ResNet_like_N_No_W2V_len50.hdfs')
Checkpoint = ModelCheckpoint(filepath,monitor='val_loss', save_best_only=True,
                             save_weights_only=True,period=1)
csv_logger = CSVLogger('./PCR_SILVA_all/W2V/ResNet_like_N_No_W2V_len50.log')
EarlyStop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

m.fit_generator(simulate_multioutput_ngs_generator(train,batch_size=batch_size,len1=50),
                epochs=100,shuffle=True,
                steps_per_epoch=train.shape[0]//batch_size,
                validation_data=simulate_multioutput_ngs_generator
                (valid,batch_size=batch_size,len1=50),
                validation_steps=valid.shape[0]//batch_size,
                verbose=2,callbacks=[Checkpoint,csv_logger,EarlyStop])

Epoch 1/100
 - 1997s - loss: 85.5805 - dense_3_loss: 12.6814 - dense_4_loss: 12.7402 - dense_5_loss: 14.9921 - dense_6_loss: 15.0239 - dense_7_loss: 15.0491 - dense_8_loss: 15.0939 - dense_3_acc: 0.2132 - dense_4_acc: 0.2096 - dense_5_acc: 0.0699 - dense_6_acc: 0.0679 - dense_7_acc: 0.0663 - dense_8_acc: 0.0636 - val_loss: 85.8867 - val_dense_3_loss: 12.7245 - val_dense_4_loss: 12.7848 - val_dense_5_loss: 15.0452 - val_dense_6_loss: 15.0792 - val_dense_7_loss: 15.1055 - val_dense_8_loss: 15.1476 - val_dense_3_acc: 0.2105 - val_dense_4_acc: 0.2068 - val_dense_5_acc: 0.0666 - val_dense_6_acc: 0.0645 - val_dense_7_acc: 0.0628 - val_dense_8_acc: 0.0603
Epoch 2/100
 - 1923s - loss: 85.5805 - dense_3_loss: 12.6814 - dense_4_loss: 12.7402 - dense_5_loss: 14.9921 - dense_6_loss: 15.0239 - dense_7_loss: 15.0491 - dense_8_loss: 15.0939 - dense_3_acc: 0.2132 - dense_4_acc: 0.2096 - dense_5_acc: 0.0699 - dense_6_acc: 0.0679 - dense_7_acc: 0.0663 - dense_8_acc: 0.0636 - val_loss: 85.8867 - val_dens

### Fixed_Len ResNet like

In [0]:
inp = Input(shape=(max_len,),dtype='uint16')
main = Embedding(max_features, vector_size, weights=[embedding_matrix],trainable=False)(inp)
#main = Embedding(max_features, 128)(inp)
#main = SpatialDropout1D(dr_rate)(main)
main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
i_l1 = MaxPooling1D(pool_size=2)(main)

main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
main = concatenate([main, i_l1],axis=1)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)

i_l1 = Conv1D(filters=128, kernel_size=1, padding='same')(main)
main = Conv1D(filters=128, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=128, kernel_size=3, padding='same')(main)
main = concatenate([main, i_l1],axis=1)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)

i_l1 = Conv1D(filters=256, kernel_size=1, padding='same')(main)
main = Conv1D(filters=256, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=256, kernel_size=3, padding='same')(main)
main = concatenate([main, i_l1],axis=1)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)

i_l1 = Conv1D(filters=512, kernel_size=1, padding='same')(main)
main = Conv1D(filters=512, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=512, kernel_size=3, padding='same')(main)
main = concatenate([main, i_l1],axis=1)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)

main = GlobalMaxPool1D()(main)
main = Dense(1024, activation='relu')(main)  # Should be 4096
main = Dense(512, activation='relu')(main)  # Should be 2048

out1 = Dense(classes_1,activation='softmax')(main)
out2 = Dense(classes_2,activation='softmax')(main)
out3 = Dense(classes_3,activation='softmax')(main)
out4 = Dense(classes_4,activation='softmax')(main)
out5 = Dense(classes_5,activation='softmax')(main)
out6 = Dense(classes_6,activation='softmax')(main)

m = Model(inputs=[inp], outputs=[out1,out2,out3,out4,out5,out6])

optimizer = AdamW(lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4, epsilon=1e-8, decay=0.)
m.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=['acc'])

In [0]:
def simulate_multiooutput_ngs_generator_fixed_len(df,len1 = 100,batch_size=250):
    y_sim_1,y_sim_2,y_sim_3,y_sim_4,y_sim_5,y_sim_6 = df['phylum-'].values,df['class_-'].values,df['order-'].values,df['family-'].values,df['genus-'].values,df['species-'].values
    x_sim = df['encoded'].apply(lambda x : simulate_reads(x,len1=len1))
    x_sim = pad_sequences(x_sim.values,maxlen=max_len)
    x_sim = array(np.concatenate(x_sim).reshape(x_sim.shape[0],max_len).tolist()).astype('uint16')
    samples_per_epoch = df.shape[0]
    number_of_batches = samples_per_epoch//batch_size
    counter=0

    while counter <=number_of_batches :
        X_batch = np.array(x_sim[batch_size*counter:batch_size*(counter+1)]).astype('uint16')
        y_1 = y_sim_1[batch_size*counter:batch_size*(counter+1)].astype('uint8')
        y_2 = y_sim_2[batch_size*counter:batch_size*(counter+1)].astype('uint8')
        y_3 = y_sim_3[batch_size*counter:batch_size*(counter+1)].astype('uint8')
        y_4 = y_sim_4[batch_size*counter:batch_size*(counter+1)].astype('uint16')
        y_5 = y_sim_5[batch_size*counter:batch_size*(counter+1)].astype('uint16')
        y_6 = y_sim_6[batch_size*counter:batch_size*(counter+1)].astype('uint16')
        counter += 1
        if counter ==number_of_batches:
            counter = 0
            del x_sim
            y_sim_1,y_sim_2,y_sim_3,y_sim_4,y_sim_5,y_sim_6 = df['phylum-'].values,df['class_-'].values,df['order-'].values,df['family-'].values,df['genus-'].values,df['species-'].values
            x_sim = df['encoded'].apply(lambda x : simulate_reads(x,len1=len1))
            x_sim = pad_sequences(x_sim.values,maxlen=max_len)
            x_sim = array(np.concatenate(x_sim).reshape(x_sim.shape[0],max_len).tolist()).astype('uint16')
        yield X_batch,[y_1,y_2,y_3,y_4,y_5,y_6]

In [0]:
batch_size = 250
m.load_weights('./PCR_SILVA_all/W2V/Fixed_len_ResNet_like_N.hdfs')
filepath = ''.join('./PCR_SILVA_all/W2V/Fixed_len_ResNet_like_N.hdfs')
Checkpoint = ModelCheckpoint(filepath,monitor='val_loss', save_best_only=True,
                             save_weights_only=True,period=1)
csv_logger = CSVLogger('./PCR_SILVA_all/W2V/Fixed_len_ResNet_like_N.log')
EarlyStop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')

train_gen = simulate_multiooutput_ngs_generator_fixed_len(train,batch_size=batch_size,len1=100)
valid_gen = simulate_multiooutput_ngs_generator_fixed_len(valid,batch_size=batch_size,len1=100)

m.fit_generator(train_gen,
                epochs=47,shuffle=True,
                steps_per_epoch=train.shape[0]//batch_size,
                validation_data=valid_gen,
                validation_steps=valid.shape[0]//batch_size,
                verbose=2,callbacks=[Checkpoint,csv_logger,EarlyStop])

Epoch 1/47
 - 1992s - loss: 1.6019 - dense_3_loss: 0.0136 - dense_4_loss: 0.0195 - dense_5_loss: 0.0465 - dense_6_loss: 0.0796 - dense_7_loss: 0.2742 - dense_8_loss: 1.1687 - dense_3_acc: 0.9975 - dense_4_acc: 0.9965 - dense_5_acc: 0.9900 - dense_6_acc: 0.9829 - dense_7_acc: 0.9300 - dense_8_acc: 0.7205 - val_loss: 2.0037 - val_dense_3_loss: 0.0185 - val_dense_4_loss: 0.0256 - val_dense_5_loss: 0.0613 - val_dense_6_loss: 0.1025 - val_dense_7_loss: 0.3234 - val_dense_8_loss: 1.4724 - val_dense_3_acc: 0.9968 - val_dense_4_acc: 0.9955 - val_dense_5_acc: 0.9874 - val_dense_6_acc: 0.9790 - val_dense_7_acc: 0.9234 - val_dense_8_acc: 0.6956
Epoch 2/47
 - 1924s - loss: 1.5654 - dense_3_loss: 0.0133 - dense_4_loss: 0.0192 - dense_5_loss: 0.0457 - dense_6_loss: 0.0774 - dense_7_loss: 0.2660 - dense_8_loss: 1.1437 - dense_3_acc: 0.9975 - dense_4_acc: 0.9966 - dense_5_acc: 0.9900 - dense_6_acc: 0.9832 - dense_7_acc: 0.9319 - dense_8_acc: 0.7226 - val_loss: 2.0026 - val_dense_3_loss: 0.0211 - val_d

### ResNet_OHE

In [0]:
inp = Input(shape=(max_len,),dtype='uint8')
#main = Embedding(max_features, vector_size, weights=[embedding_matrix],trainable=False)(inp)
main = Embedding(5, 128)(inp)
#main = SpatialDropout1D(dr_rate)(main)
main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
i_l1 = MaxPooling1D(pool_size=2)(main)

main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
main = concatenate([main, i_l1],axis=1)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)

i_l1 = Conv1D(filters=128, kernel_size=1, padding='same')(main)
main = Conv1D(filters=128, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=128, kernel_size=3, padding='same')(main)
main = concatenate([main, i_l1],axis=1)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)

i_l1 = Conv1D(filters=256, kernel_size=1, padding='same')(main)
main = Conv1D(filters=256, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=256, kernel_size=3, padding='same')(main)
main = concatenate([main, i_l1],axis=1)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)

i_l1 = Conv1D(filters=512, kernel_size=1, padding='same')(main)
main = Conv1D(filters=512, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=512, kernel_size=3, padding='same')(main)
main = concatenate([main, i_l1],axis=1)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)

main = GlobalMaxPool1D()(main)
main = Dense(1024, activation='relu')(main)  # Should be 4096
main = Dense(512, activation='relu')(main)  # Should be 2048

out1 = Dense(classes_1,activation='softmax')(main)
out2 = Dense(classes_2,activation='softmax')(main)
out3 = Dense(classes_3,activation='softmax')(main)
out4 = Dense(classes_4,activation='softmax')(main)
out5 = Dense(classes_5,activation='softmax')(main)
out6 = Dense(classes_6,activation='softmax')(main)

m = Model(inputs=[inp], outputs=[out1,out2,out3,out4,out5,out6])

optimizer = AdamW(lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4, epsilon=1e-8, decay=0.)
m.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=['acc'])

In [0]:
batch_size = 250
#m.load_weights('./PCR_SILVA_all/W2V/ResNet_like_N_OHE_len50.hdfs')
filepath = ''.join('./PCR_SILVA_all/W2V/ResNet_like_N_OHE_len50.hdfs')
Checkpoint = ModelCheckpoint(filepath,monitor='val_loss', save_best_only=True,
                             save_weights_only=True,period=1)
csv_logger = CSVLogger('./PCR_SILVA_all/W2V/ResNet_like_N_OHE_len50.log')
EarlyStop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

m.fit_generator(simulate_multioutput_ngs_generator(train,batch_size=batch_size,len1=50),
                epochs=50,shuffle=True,
                steps_per_epoch=train.shape[0]//batch_size,
                validation_data=simulate_multioutput_ngs_generator
                (valid,batch_size=batch_size,len1=50),
                validation_steps=valid.shape[0]//batch_size,
                verbose=2,callbacks=[Checkpoint,csv_logger,EarlyStop])

### 2-Block ResNet

In [0]:
inp = Input(shape=(max_len,),dtype='uint16')
main = Embedding(max_features, vector_size, weights=[embedding_matrix],trainable=False)(inp)
#main = Embedding(5, 128)(inp)
#main = SpatialDropout1D(dr_rate)(main)
main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
i_l1 = MaxPooling1D(pool_size=2)(main)

main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
main = concatenate([main, i_l1],axis=1)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)

i_l1 = Conv1D(filters=128, kernel_size=1, padding='same')(main)
main = Conv1D(filters=128, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=128, kernel_size=3, padding='same')(main)
main = concatenate([main, i_l1],axis=1)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)


main = GlobalMaxPool1D()(main)
main = Dense(1024, activation='relu')(main)  # Should be 4096
main = Dense(512, activation='relu')(main)  # Should be 2048

out1 = Dense(classes_1,activation='softmax')(main)
out2 = Dense(classes_2,activation='softmax')(main)
out3 = Dense(classes_3,activation='softmax')(main)
out4 = Dense(classes_4,activation='softmax')(main)
out5 = Dense(classes_5,activation='softmax')(main)
out6 = Dense(classes_6,activation='softmax')(main)

m = Model(inputs=[inp], outputs=[out1,out2,out3,out4,out5,out6])

optimizer = AdamW(lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4, epsilon=1e-8, decay=0.)
m.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=['acc'])

In [0]:
batch_size = 250
#m.load_weights('./PCR_SILVA_all/W2V/ResNet_like_N_big.hdfs')
filepath = ''.join('./PCR_SILVA_all/W2V/2_Block_ResNet.hdfs')
Checkpoint = ModelCheckpoint(filepath,monitor='val_loss', save_best_only=True,
                             save_weights_only=True,period=1)
csv_logger = CSVLogger('./PCR_SILVA_all/W2V/2_Block_ResNet.log')
EarlyStop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

m.fit_generator(simulate_multioutput_ngs_generator(train,batch_size=batch_size,len1=25),
                epochs=50,shuffle=True,
                steps_per_epoch=train.shape[0]//batch_size,
                validation_data=simulate_multioutput_ngs_generator
                (valid,batch_size=batch_size,len1=25),
                validation_steps=valid.shape[0]//batch_size,
                verbose=2,callbacks=[Checkpoint,csv_logger,EarlyStop])

### 2_Block_ResNet_No_W2V

In [0]:
#2_Block_ResNet_No_W2V
inp = Input(shape=(max_len,),dtype='uint16')
#main = Embedding(max_features, vector_size, weights=[embedding_matrix],trainable=False)(inp)
main = Embedding(5, 128)(inp)
#main = SpatialDropout1D(dr_rate)(main)
main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
i_l1 = MaxPooling1D(pool_size=2)(main)

main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
main = concatenate([main, i_l1],axis=1)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)

i_l1 = Conv1D(filters=128, kernel_size=1, padding='same')(main)
main = Conv1D(filters=128, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=128, kernel_size=3, padding='same')(main)
main = concatenate([main, i_l1],axis=1)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)


main = GlobalMaxPool1D()(main)
main = Dense(1024, activation='relu')(main)  # Should be 4096
main = Dense(512, activation='relu')(main)  # Should be 2048

out1 = Dense(classes_1,activation='softmax')(main)
out2 = Dense(classes_2,activation='softmax')(main)
out3 = Dense(classes_3,activation='softmax')(main)
out4 = Dense(classes_4,activation='softmax')(main)
out5 = Dense(classes_5,activation='softmax')(main)
out6 = Dense(classes_6,activation='softmax')(main)

m = Model(inputs=[inp], outputs=[out1,out2,out3,out4,out5,out6])

optimizer = AdamW(lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4, epsilon=1e-8, decay=0.)
m.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=['acc'])

In [0]:
batch_size = 250
#m.load_weights('./PCR_SILVA_all/W2V/2_Block_ResNet_No_W2V.hdfs')
filepath = ''.join('./PCR_SILVA_all/W2V/2_Block_ResNet.hdfs')
Checkpoint = ModelCheckpoint(filepath,monitor='val_loss', save_best_only=True,
                             save_weights_only=True,period=1)
csv_logger = CSVLogger('./PCR_SILVA_all/W2V/2_Block_ResNet_No_W2V.log')
EarlyStop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

m.fit_generator(simulate_multioutput_ngs_generator(train,batch_size=batch_size,len1=25),
                epochs=50,shuffle=True,
                steps_per_epoch=train.shape[0]//batch_size,
                validation_data=simulate_multioutput_ngs_generator
                (valid,batch_size=batch_size,len1=25),
                validation_steps=valid.shape[0]//batch_size,
                verbose=2,callbacks=[Checkpoint,csv_logger,EarlyStop])

### 6_block_ResNet_No_W2V

In [0]:
#6_Block_ResNet_No_W2V
inp = Input(shape=(max_len,),dtype='uint16')
#main = Embedding(max_features, vector_size, weights=[embedding_matrix],trainable=False)(inp)
main = Embedding(5, 128)(inp)
#main = SpatialDropout1D(dr_rate)(main)
main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
i_l1 = MaxPooling1D(pool_size=2)(main)

main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
main = concatenate([main, i_l1],axis=1)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)

for i in range(5):
  i_l1 = Conv1D(filters=128, kernel_size=1, padding='same')(main)
  main = Conv1D(filters=128, kernel_size=3, padding='same')(main)
  main = BatchNormalization()(main)
  main = Activation('relu')(main)
  main = Conv1D(filters=128, kernel_size=3, padding='same')(main)
  main = concatenate([main, i_l1],axis=1)
  main = BatchNormalization()(main)
  main = Activation('relu')(main)
  main = MaxPooling1D(pool_size=2)(main)

main = GlobalMaxPool1D()(main)
main = Dense(2048, activation='relu')(main)  # Should be 4096
main = Dense(1024, activation='relu')(main)  # Should be 2048

out1 = Dense(classes_1,activation='softmax')(main)
out2 = Dense(classes_2,activation='softmax')(main)
out3 = Dense(classes_3,activation='softmax')(main)
out4 = Dense(classes_4,activation='softmax')(main)
out5 = Dense(classes_5,activation='softmax')(main)
out6 = Dense(classes_6,activation='softmax')(main)

m = Model(inputs=[inp], outputs=[out1,out2,out3,out4,out5,out6])

optimizer = AdamW(lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4, epsilon=1e-8, decay=0.)
m.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=['acc'])

In [0]:
batch_size = 250
#m.load_weights('./PCR_SILVA_all/W2V/6_Block_ResNet_No_W2V.hdfs')
filepath = ''.join('./PCR_SILVA_all/W2V/6_Block_ResNet_No_W2V.hdfs')
Checkpoint = ModelCheckpoint(filepath,monitor='val_loss', save_best_only=True,
                             save_weights_only=True,period=1)
csv_logger = CSVLogger('./PCR_SILVA_all/W2V/2_Block_ResNet_No_W2V.log')
EarlyStop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

m.fit_generator(simulate_multioutput_ngs_generator(train,batch_size=batch_size,len1=25),
                epochs=50,shuffle=True,
                steps_per_epoch=train.shape[0]//batch_size,
                validation_data=simulate_multioutput_ngs_generator
                (valid,batch_size=batch_size,len1=25),
                validation_steps=valid.shape[0]//batch_size,
                verbose=2,callbacks=[Checkpoint,csv_logger,EarlyStop])

Epoch 1/50
 - 1324s - loss: 20.6310 - dense_35_loss: 1.3044 - dense_36_loss: 1.8062 - dense_37_loss: 2.8904 - dense_38_loss: 3.6665 - dense_39_loss: 4.9024 - dense_40_loss: 6.0611 - dense_35_acc: 0.3955 - dense_36_acc: 0.3094 - dense_37_acc: 0.2145 - dense_38_acc: 0.1489 - dense_39_acc: 0.1380 - dense_40_acc: 0.1262 - val_loss: 20.6626 - val_dense_35_loss: 1.2920 - val_dense_36_loss: 1.7810 - val_dense_37_loss: 2.8783 - val_dense_38_loss: 3.6469 - val_dense_39_loss: 4.8869 - val_dense_40_loss: 6.1775 - val_dense_35_acc: 0.3970 - val_dense_36_acc: 0.3252 - val_dense_37_acc: 0.2257 - val_dense_38_acc: 0.1587 - val_dense_39_acc: 0.1396 - val_dense_40_acc: 0.1247
Epoch 2/50
 - 1246s - loss: 20.3635 - dense_35_loss: 1.2877 - dense_36_loss: 1.7764 - dense_37_loss: 2.8596 - dense_38_loss: 3.6225 - dense_39_loss: 4.8442 - dense_40_loss: 5.9731 - dense_35_acc: 0.3972 - dense_36_acc: 0.3223 - dense_37_acc: 0.2257 - dense_38_acc: 0.1597 - dense_39_acc: 0.1405 - dense_40_acc: 0.1284 - val_loss: 20

### 6_block_ResNet

In [0]:
#6_Block_ResNet_No_W2V
inp = Input(shape=(max_len,),dtype='uint16')
main = Embedding(max_features, vector_size, weights=[embedding_matrix],trainable=False)(inp)
#main = Embedding(5, 128)(inp)
#main = SpatialDropout1D(dr_rate)(main)
main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
i_l1 = MaxPooling1D(pool_size=2)(main)

main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = Conv1D(filters=64, kernel_size=3, padding='same')(main)
main = concatenate([main, i_l1],axis=1)
main = BatchNormalization()(main)
main = Activation('relu')(main)
main = MaxPooling1D(pool_size=2)(main)

for i in range(5):
    i_l1 = Conv1D(filters=128, kernel_size=1, padding='same')(main)
    main = Conv1D(filters=128, kernel_size=3, padding='same')(main)
    main = BatchNormalization()(main)
    main = Activation('relu')(main)
    main = Conv1D(filters=128, kernel_size=3, padding='same')(main)
    main = concatenate([main, i_l1],axis=1)
    main = BatchNormalization()(main)
    main = Activation('relu')(main)
    main = MaxPooling1D(pool_size=2)(main)

main = GlobalMaxPool1D()(main)
main = Dense(2048, activation='relu')(main)  # Should be 4096
main = Dense(1024, activation='relu')(main)  # Should be 2048

out1 = Dense(classes_1,activation='softmax')(main)
out2 = Dense(classes_2,activation='softmax')(main)
out3 = Dense(classes_3,activation='softmax')(main)
out4 = Dense(classes_4,activation='softmax')(main)
out5 = Dense(classes_5,activation='softmax')(main)
out6 = Dense(classes_6,activation='softmax')(main)

m = Model(inputs=[inp], outputs=[out1,out2,out3,out4,out5,out6])

optimizer = AdamW(lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4, epsilon=1e-8, decay=0.)
m.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=['acc'])

In [0]:
batch_size = 250
#m.load_weights('./PCR_SILVA_all/W2V/6_Block_ResNet_No_W2V.hdfs')
filepath = ''.join('./PCR_SILVA_all/W2V/6_Block_ResNet.hdfs')
Checkpoint = ModelCheckpoint(filepath,monitor='val_loss', save_best_only=True,
                             save_weights_only=True,period=1)
csv_logger = CSVLogger('./PCR_SILVA_all/W2V/2_Block_ResNet.log')
EarlyStop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

m.fit_generator(simulate_multioutput_ngs_generator(train,batch_size=batch_size,len1=25),
                epochs=50,shuffle=True,
                steps_per_epoch=train.shape[0]//batch_size,
                validation_data=simulate_multioutput_ngs_generator
                (valid,batch_size=batch_size,len1=25),
                validation_steps=valid.shape[0]//batch_size,
                verbose=2,callbacks=[Checkpoint,csv_logger,EarlyStop])

Epoch 1/50
 - 1297s - loss: 7.2411 - dense_3_loss: 0.2169 - dense_4_loss: 0.3219 - dense_5_loss: 0.6687 - dense_6_loss: 0.9779 - dense_7_loss: 1.8557 - dense_8_loss: 3.2000 - dense_3_acc: 0.9288 - dense_4_acc: 0.9031 - dense_5_acc: 0.8217 - dense_6_acc: 0.7601 - dense_7_acc: 0.5991 - dense_8_acc: 0.4473 - val_loss: 6.5384 - val_dense_3_loss: 0.1770 - val_dense_4_loss: 0.2784 - val_dense_5_loss: 0.5657 - val_dense_6_loss: 0.8243 - val_dense_7_loss: 1.5830 - val_dense_8_loss: 3.1099 - val_dense_3_acc: 0.9405 - val_dense_4_acc: 0.9134 - val_dense_5_acc: 0.8439 - val_dense_6_acc: 0.7895 - val_dense_7_acc: 0.6400 - val_dense_8_acc: 0.4644
Epoch 2/50
 - 1228s - loss: 5.2040 - dense_3_loss: 0.1464 - dense_4_loss: 0.2192 - dense_5_loss: 0.4500 - dense_6_loss: 0.6509 - dense_7_loss: 1.2855 - dense_8_loss: 2.4520 - dense_3_acc: 0.9472 - dense_4_acc: 0.9296 - dense_5_acc: 0.8743 - dense_6_acc: 0.8328 - dense_7_acc: 0.6996 - dense_8_acc: 0.5225 - val_loss: 5.6615 - val_dense_3_loss: 0.1580 - val_d

### MLP_W2V

In [0]:
#inp = Input(shape=(max_len,), dtype='uint8')
inp = Input(shape=(max_len,),dtype='uint16')
main = Embedding(max_features, vector_size, weights=[embedding_matrix],trainable=False)(inp)
#main = Embedding(max_features, 128)(inp)
main = Dense(512)(main)
#main = Dropout(0.25)(main)
#main = Dense(128)(main)
main = GlobalMaxPool1D()(main)
main = Dropout(0.25)(main)
main = Dense(512)(main)  # Should be 4096
main = Dropout(0.25)(main)
#main = Dense(256, activation='relu')(main)  # Should be 2048

out1 = Dense(classes_1,activation='softmax')(main)
out2 = Dense(classes_2,activation='softmax')(main)
out3 = Dense(classes_3,activation='softmax')(main)
out4 = Dense(classes_4,activation='softmax')(main)
out5 = Dense(classes_5,activation='softmax')(main)
out6 = Dense(classes_6,activation='softmax')(main)

m = Model(inputs=[inp], outputs=[out1,out2,out3,out4,out5,out6])

optimizer = AdamW(lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4, epsilon=1e-8, decay=0.)
m.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=['acc'])

In [0]:
batch_size = 250
m.load_weights('./PCR_SILVA_all/W2V/MLP_W2V_len50.hdfs')
filepath = ''.join('./PCR_SILVA_all/W2V/MLP_W2V_len50.hdfs')
Checkpoint = ModelCheckpoint(filepath,monitor='val_loss', save_best_only=True,
                             save_weights_only=True,period=1)
csv_logger = CSVLogger('./PCR_SILVA_all/W2V/MLP_W2V.log')
EarlyStop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

m.fit_generator(simulate_multioutput_ngs_generator(train,batch_size=batch_size,len1=50),
                epochs=100,shuffle=True,
                steps_per_epoch=train.shape[0]//batch_size,
                validation_data=simulate_multioutput_ngs_generator
                (valid,batch_size=batch_size,len1=50),
                validation_steps=valid.shape[0]//batch_size,
                verbose=2,callbacks=[Checkpoint,csv_logger,EarlyStop ])
# +8

Epoch 1/100
 - 312s - loss: 2.7518 - dense_19_loss: 0.0525 - dense_20_loss: 0.0752 - dense_21_loss: 0.1650 - dense_22_loss: 0.2412 - dense_23_loss: 0.5764 - dense_24_loss: 1.6416 - dense_19_acc: 0.9862 - dense_20_acc: 0.9798 - dense_21_acc: 0.9560 - dense_22_acc: 0.9400 - dense_23_acc: 0.8551 - dense_24_acc: 0.6506 - val_loss: 2.7946 - val_dense_19_loss: 0.0425 - val_dense_20_loss: 0.0614 - val_dense_21_loss: 0.1449 - val_dense_22_loss: 0.2148 - val_dense_23_loss: 0.5380 - val_dense_24_loss: 1.7931 - val_dense_19_acc: 0.9894 - val_dense_20_acc: 0.9842 - val_dense_21_acc: 0.9621 - val_dense_22_acc: 0.9477 - val_dense_23_acc: 0.8675 - val_dense_24_acc: 0.6457
Epoch 2/100
 - 239s - loss: 2.6591 - dense_19_loss: 0.0502 - dense_20_loss: 0.0735 - dense_21_loss: 0.1635 - dense_22_loss: 0.2371 - dense_23_loss: 0.5646 - dense_24_loss: 1.5701 - dense_19_acc: 0.9867 - dense_20_acc: 0.9803 - dense_21_acc: 0.9563 - dense_22_acc: 0.9405 - dense_23_acc: 0.8564 - dense_24_acc: 0.6530 - val_loss: 2.742

### MLP_no_W2V

In [0]:
inp = Input(shape=(max_len,),dtype='uint16')
#main = Embedding(max_features, vector_size, weights=[embedding_matrix],trainable=False)(inp)
main = Embedding(max_features, 128)(inp)
#main = Embedding(max_features, 128)(inp)
main = Dense(512)(main)
#main = Dropout(0.25)(main)
#main = Dense(128)(main)
main = GlobalMaxPool1D()(main)
main = Dropout(0.25)(main)
main = Dense(512)(main)  # Should be 4096
main = Dropout(0.25)(main)

out1 = Dense(classes_1,activation='softmax')(main)
out2 = Dense(classes_2,activation='softmax')(main)
out3 = Dense(classes_3,activation='softmax')(main)
out4 = Dense(classes_4,activation='softmax')(main)
out5 = Dense(classes_5,activation='softmax')(main)
out6 = Dense(classes_6,activation='softmax')(main)

m = Model(inputs=[inp], outputs=[out1,out2,out3,out4,out5,out6])

optimizer = AdamW(lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4, epsilon=1e-8, decay=0.)
m.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=['acc'])

In [0]:
batch_size = 250
#m.load_weights('./PCR_SILVA_all/W2V/MLP_no_W2V_len50.hdfs')
filepath = ''.join('./PCR_SILVA_all/W2V/MLP_no_W2V_len50_fresh.hdfs')
Checkpoint = ModelCheckpoint(filepath,monitor='val_loss', save_best_only=True,
                             save_weights_only=True,period=1)
csv_logger = CSVLogger('./PCR_SILVA_all/W2V/MLP_no_W2V_len50_fresh.log')
EarlyStop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

m.fit_generator(simulate_multioutput_ngs_generator(train,batch_size=batch_size,len1=50),
                epochs=100,shuffle=True,
                steps_per_epoch=train.shape[0]//batch_size,
                validation_data=simulate_multioutput_ngs_generator
                (valid,batch_size=batch_size,len1=50),
                validation_steps=valid.shape[0]//batch_size,
                verbose=2,callbacks=[Checkpoint,csv_logger,EarlyStop])

Epoch 1/100
 - 344s - loss: 4.8768 - dense_19_loss: 0.1127 - dense_20_loss: 0.1626 - dense_21_loss: 0.3604 - dense_22_loss: 0.5396 - dense_23_loss: 1.1729 - dense_24_loss: 2.5286 - dense_19_acc: 0.9711 - dense_20_acc: 0.9584 - dense_21_acc: 0.9110 - dense_22_acc: 0.8756 - dense_23_acc: 0.7486 - dense_24_acc: 0.5546 - val_loss: 3.2914 - val_dense_19_loss: 0.0484 - val_dense_20_loss: 0.0693 - val_dense_21_loss: 0.1728 - val_dense_22_loss: 0.2678 - val_dense_23_loss: 0.6754 - val_dense_24_loss: 2.0577 - val_dense_19_acc: 0.9883 - val_dense_20_acc: 0.9825 - val_dense_21_acc: 0.9561 - val_dense_22_acc: 0.9349 - val_dense_23_acc: 0.8381 - val_dense_24_acc: 0.6154
Epoch 2/100
 - 262s - loss: 3.0024 - dense_19_loss: 0.0545 - dense_20_loss: 0.0778 - dense_21_loss: 0.1808 - dense_22_loss: 0.2695 - dense_23_loss: 0.6572 - dense_24_loss: 1.7625 - dense_19_acc: 0.9862 - dense_20_acc: 0.9798 - dense_21_acc: 0.9531 - dense_22_acc: 0.9343 - dense_23_acc: 0.8386 - dense_24_acc: 0.6307 - val_loss: 2.976

### MLP_ No_Kmer

In [0]:
inp = Input(shape=(max_len,),dtype='uint8')
#main = Embedding(max_features, vector_size, weights=[embedding_matrix],trainable=False)(inp)
main = Embedding(5, 128)(inp)
#main = Embedding(max_features, 128)(inp)
main = Dense(512)(main)
#main = Dropout(0.25)(main)
#main = Dense(128)(main)
main = GlobalMaxPool1D()(main)
main = Dropout(0.25)(main)
main = Dense(512)(main)  # Should be 4096
main = Dropout(0.25)(main)

out1 = Dense(classes_1,activation='softmax')(main)
out2 = Dense(classes_2,activation='softmax')(main)
out3 = Dense(classes_3,activation='softmax')(main)
out4 = Dense(classes_4,activation='softmax')(main)
out5 = Dense(classes_5,activation='softmax')(main)
out6 = Dense(classes_6,activation='softmax')(main)

m = Model(inputs=[inp], outputs=[out1,out2,out3,out4,out5,out6])

optimizer = AdamW(lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4, epsilon=1e-8, decay=0.)
m.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=['acc'])

In [0]:
batch_size = 250
#m.load_weights('./PCR_SILVA_all/W2V/ResNet_like_N.hdfs')
filepath = ''.join('./PCR_SILVA_all/W2V/MLP_no_W2V_no_kmer_len50.hdfs')
Checkpoint = ModelCheckpoint(filepath,monitor='val_loss', save_best_only=True,
                             save_weights_only=True,period=1)
csv_logger = CSVLogger('./PCR_SILVA_all/W2V/MLP_no_W2V_no_kmer_len50.log')
EarlyStop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

m.fit_generator(simulate_multioutput_ngs_generator(train,batch_size=batch_size,len1=50),
                epochs=100,shuffle=True,
                steps_per_epoch=train.shape[0]//batch_size,
                validation_data=simulate_multioutput_ngs_generator
                (valid,batch_size=batch_size,len1=50),
                validation_steps=valid.shape[0]//batch_size,
                verbose=2,callbacks=[Checkpoint,csv_logger,EarlyStop])

Epoch 1/100
 - 294s - loss: 20.8199 - dense_27_loss: 1.3207 - dense_28_loss: 1.8366 - dense_29_loss: 2.9243 - dense_30_loss: 3.7074 - dense_31_loss: 4.9392 - dense_32_loss: 6.0916 - dense_27_acc: 0.3928 - dense_28_acc: 0.2948 - dense_29_acc: 0.2014 - dense_30_acc: 0.1366 - dense_31_acc: 0.1360 - dense_32_acc: 0.1241 - val_loss: 20.9718 - val_dense_27_loss: 1.3127 - val_dense_28_loss: 1.8260 - val_dense_29_loss: 2.9240 - val_dense_30_loss: 3.7116 - val_dense_31_loss: 4.9529 - val_dense_32_loss: 6.2445 - val_dense_27_acc: 0.3966 - val_dense_28_acc: 0.3033 - val_dense_29_acc: 0.2037 - val_dense_30_acc: 0.1376 - val_dense_31_acc: 0.1363 - val_dense_32_acc: 0.1211
Epoch 2/100
 - 239s - loss: 20.6903 - dense_27_loss: 1.3104 - dense_28_loss: 1.8239 - dense_29_loss: 2.9081 - dense_30_loss: 3.6902 - dense_31_loss: 4.9159 - dense_32_loss: 6.0417 - dense_27_acc: 0.3985 - dense_28_acc: 0.2995 - dense_29_acc: 0.2030 - dense_30_acc: 0.1375 - dense_31_acc: 0.1362 - dense_32_acc: 0.1246 - val_loss: 20

### Fixed_len MLP

In [0]:
inp = Input(shape=(max_len,),dtype='uint16')
#main = Embedding(max_features, vector_size, weights=[embedding_matrix],trainable=False)(inp)
main = Embedding(5, 128)(inp)
#main = Embedding(max_features, 128)(inp)
main = Dense(512)(main)
#main = Dropout(0.25)(main)
#main = Dense(128)(main)
main = GlobalMaxPool1D()(main)
main = Dropout(0.25)(main)
main = Dense(512)(main)  # Should be 4096
main = Dropout(0.25)(main)

out1 = Dense(classes_1,activation='softmax')(main)
out2 = Dense(classes_2,activation='softmax')(main)
out3 = Dense(classes_3,activation='softmax')(main)
out4 = Dense(classes_4,activation='softmax')(main)
out5 = Dense(classes_5,activation='softmax')(main)
out6 = Dense(classes_6,activation='softmax')(main)

m = Model(inputs=[inp], outputs=[out1,out2,out3,out4,out5,out6])

optimizer = AdamW(lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4, epsilon=1e-8, decay=0.)
m.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=['acc'])

In [0]:
def simulate_multiooutput_ngs_generator_fixed_len(df,len1 = 100,batch_size=250):
    y_sim_1,y_sim_2,y_sim_3,y_sim_4,y_sim_5,y_sim_6 = df['phylum-'].values,df['class_-'].values,df['order-'].values,df['family-'].values,df['genus-'].values,df['species-'].values
    x_sim = df['encoded'].apply(lambda x : simulate_reads_range(x,min_len=len1))
    x_sim = pad_sequences(x_sim.values,maxlen=max_len)
    x_sim = array(np.concatenate(x_sim).reshape(x_sim.shape[0],max_len).tolist()).astype('uint16')
    samples_per_epoch = df.shape[0]
    number_of_batches = samples_per_epoch//batch_size
    counter=0

    while counter <=number_of_batches :
        X_batch = np.array(x_sim[batch_size*counter:batch_size*(counter+1)]).astype('uint16')
        y_1 = y_sim_1[batch_size*counter:batch_size*(counter+1)].astype('uint8')
        y_2 = y_sim_2[batch_size*counter:batch_size*(counter+1)].astype('uint8')
        y_3 = y_sim_3[batch_size*counter:batch_size*(counter+1)].astype('uint8')
        y_4 = y_sim_4[batch_size*counter:batch_size*(counter+1)].astype('uint16')
        y_5 = y_sim_5[batch_size*counter:batch_size*(counter+1)].astype('uint16')
        y_6 = y_sim_6[batch_size*counter:batch_size*(counter+1)].astype('uint16')
        counter += 1
        if counter ==number_of_batches:
            counter = 0
            del x_sim
            y_sim_1,y_sim_2,y_sim_3,y_sim_4,y_sim_5,y_sim_6 = df['phylum-'].values,df['class_-'].values,df['order-'].values,df['family-'].values,df['genus-'].values,df['species-'].values
            x_sim = df['encoded'].apply(lambda x : simulate_reads_range(x,min_len=len1))
            x_sim = pad_sequences(x_sim.values,maxlen=max_len)
            x_sim = array(np.concatenate(x_sim).reshape(x_sim.shape[0],max_len).tolist()).astype('uint16')
        yield X_batch,[y_1,y_2,y_3,y_4,y_5,y_6]

In [0]:
batch_size = 250
#m.load_weights('./PCR_SILVA_all/W2V/ResNet_like_N.hdfs')
filepath = ''.join('./PCR_SILVA_all/W2V/Fixed_len_MLP_No_W2V.hdfs')
Checkpoint = ModelCheckpoint(filepath,monitor='val_loss', save_best_only=True,
                             save_weights_only=True,period=1)
csv_logger = CSVLogger('./PCR_SILVA_all/W2V/Fixed_len_MLP_No_W2V.log')
EarlyStop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

train_gen = simulate_multiooutput_ngs_generator_fixed_len(train,batch_size=batch_size,len1=100)
valid_gen = simulate_multiooutput_ngs_generator_fixed_len(valid,batch_size=batch_size,len1=100)

m.fit_generator(train_gen,
                epochs=100,shuffle=True,
                steps_per_epoch=train.shape[0]//batch_size,
                validation_data=valid_gen,
                validation_steps=valid.shape[0]//batch_size,
                verbose=2,callbacks=[Checkpoint,csv_logger,EarlyStop])

Epoch 1/100
 - 331s - loss: 80.8092 - dense_27_loss: 10.6429 - dense_28_loss: 11.2965 - dense_29_loss: 14.8716 - dense_30_loss: 15.0022 - dense_31_loss: 13.9717 - dense_32_loss: 15.0243 - dense_27_acc: 0.3384 - dense_28_acc: 0.2978 - dense_29_acc: 0.0758 - dense_30_acc: 0.0678 - dense_31_acc: 0.1318 - dense_32_acc: 0.0668 - val_loss: 80.7294 - val_dense_27_loss: 10.5845 - val_dense_28_loss: 11.2283 - val_dense_29_loss: 14.8600 - val_dense_30_loss: 15.0790 - val_dense_31_loss: 13.9220 - val_dense_32_loss: 15.0556 - val_dense_27_acc: 0.3433 - val_dense_28_acc: 0.3034 - val_dense_29_acc: 0.0781 - val_dense_30_acc: 0.0645 - val_dense_31_acc: 0.1363 - val_dense_32_acc: 0.0660
Epoch 2/100
 - 253s - loss: 81.2970 - dense_27_loss: 10.6651 - dense_28_loss: 11.2850 - dense_29_loss: 14.8910 - dense_30_loss: 15.0239 - dense_31_loss: 13.9224 - dense_32_loss: 15.5096 - dense_27_acc: 0.3383 - dense_28_acc: 0.2999 - dense_29_acc: 0.0761 - dense_30_acc: 0.0679 - dense_31_acc: 0.1362 - dense_32_acc: 0.0

### Multioutput Evaluation

In [0]:
def simulate_multiooutput_ngs_generator_fixed_len(df,len1 = 100,batch_size=250):
    y_sim_1,y_sim_2,y_sim_3,y_sim_4,y_sim_5,y_sim_6 = df['phylum-'].values,df['class_-'].values,df['order-'].values,df['family-'].values,df['genus-'].values,df['species-'].values
    x_sim = df['encoded'].apply(lambda x : simulate_reads(x,len1=len1))
    x_sim = pad_sequences(x_sim.values,maxlen=max_len)
    x_sim = array(np.concatenate(x_sim).reshape(x_sim.shape[0],max_len).tolist()).astype('uint16')
    samples_per_epoch = df.shape[0]
    number_of_batches = samples_per_epoch//batch_size
    counter=0

    while counter <=number_of_batches :
        X_batch = np.array(x_sim[batch_size*counter:batch_size*(counter+1)]).astype('uint16')
        y_1 = y_sim_1[batch_size*counter:batch_size*(counter+1)].astype('uint8')
        y_2 = y_sim_2[batch_size*counter:batch_size*(counter+1)].astype('uint8')
        y_3 = y_sim_3[batch_size*counter:batch_size*(counter+1)].astype('uint8')
        y_4 = y_sim_4[batch_size*counter:batch_size*(counter+1)].astype('uint16')
        y_5 = y_sim_5[batch_size*counter:batch_size*(counter+1)].astype('uint16')
        y_6 = y_sim_6[batch_size*counter:batch_size*(counter+1)].astype('uint16')
        counter += 1
        if counter ==number_of_batches:
            counter = 0
            del x_sim
            y_sim_1,y_sim_2,y_sim_3,y_sim_4,y_sim_5,y_sim_6 = df['phylum-'].values,df['class_-'].values,df['order-'].values,df['family-'].values,df['genus-'].values,df['species-'].values
            x_sim = df['encoded'].apply(lambda x : simulate_reads(x,len1=len1))
            x_sim = pad_sequences(x_sim.values,maxlen=max_len)
            x_sim = array(np.concatenate(x_sim).reshape(x_sim.shape[0],max_len).tolist()).astype('uint8')
        yield X_batch,[y_1,y_2,y_3,y_4,y_5,y_6]

In [0]:
def evaluate_multioutput_on_different_length(model, lengths,data,batch_size,save_path):
    eval_df =  pd.DataFrame(np.zeros((6,len(lengths))),columns=lengths,index=['Phylum','Order','Class','Family','Genus','Species'])
    for len_ in range(len(lengths[:-1])):
        eval_df.iloc[:,len_] = model.evaluate_generator(simulate_multiooutput_ngs_generator_fixed_len(data,len1=lengths[len_],batch_size=batch_size),
                                                                                                verbose=2,steps=data.shape[0]//batch_size)[-6:]
        eval_df.to_csv(save_path)  
    data['simulated'] = pad_sequences(data['encoded'].values,maxlen=max_len).tolist()
    y_sim_1,y_sim_2,y_sim_3,y_sim_4,y_sim_5,y_sim_6 = data['phylum-'].values,data['class_-'].values,data['order-'].values,data['family-'].values,data['genus-'].values,data['species-'].values
    x_sim = data['simulated']
    x_sim = array(np.concatenate(x_sim.values).reshape(x_sim.shape[0],max_len).tolist()).astype('uint16')
    eval_df.iloc[:,-1]= m.evaluate(x_sim,[y_sim_1,y_sim_2,y_sim_3,y_sim_4,y_sim_5,y_sim_6 ],batch_size=250)[-6:]
    eval_df.to_csv(save_path)
    return eval_df

### ResNet Like network evaluation

In [0]:
m.load_weights('./PCR_SILVA_all/W2V/ResNet_like_N__.hdfs')
evaluate_multioutput_on_different_length(model=m, lengths=[25,50,75,100,125,'full_HVR'],data=test,
                                         batch_size=250,save_path='./PCR_SILVA_all/ResNet_like_N__.csv')

174500/174500 [==============================] - 181s 1ms/step


,25,50,75,100,125,full_HVR
Phylum,0.958934,0.992115,0.996762,0.997628,0.998046,0.998430
Order,0.946693,0.987152,0.994894,0.996372,0.997192,0.997713
Class,0.907725,0.965656,0.983616,0.988699,0.992361,0.994017
Family,0.883198,0.951066,0.974682,0.981593,0.987278,0.989662
Genus,0.789077,0.867169,0.910052,0.930562,0.946057,0.954338
Species,0.599146,0.656842,0.692080,0.709731,0.722986,0.732246


In [0]:
m.load_weights('./PCR_SILVA_all/W2V/ResNet_like_N_2.hdfs')
evaluate_multioutput_on_different_length(model=m, lengths=[25,50,75,100,125,'full_HVR'],data=test,
                                         batch_size=250,save_path='./PCR_SILVA_all/ResNet_like_N_2.csv')

174500/174500 [==============================] - 54s 309us/step


,25,50,75,100,125,full_HVR
Phylum,0.505415,0.511381,0.513656,0.512327,0.515209,0.998052
Order,0.313352,0.309009,0.308178,0.310556,0.319077,0.997186
Class,0.196292,0.196298,0.195307,0.191043,0.189542,0.992871
Family,0.137771,0.139135,0.137060,0.133484,0.130745,0.988212
Genus,0.061547,0.056378,0.050630,0.046103,0.042092,0.951805
Species,0.050688,0.045026,0.040143,0.035748,0.032241,0.732797


In [0]:
m.load_weights('./PCR_SILVA_all/W2V/ResNet_like_N_2_no_W2V.hdfs')
evaluate_multioutput_on_different_length(model=m, lengths=[25,50,75,100,125,'full_HVR'],data=test,
                                         batch_size=250,save_path='./PCR_SILVA_all/ResNet_like_N_2_no_W2V.csv')

174500/174500 [==============================] - 54s 310us/step


,25,50,75,100,125,full_HVR
Phylum,0.403077,0.402802,0.404648,0.402579,0.403840,0.418539
Order,0.314046,0.336968,0.364418,0.382011,0.402991,0.439897
Class,0.173163,0.181610,0.189221,0.200321,0.212355,0.216441
Family,0.118235,0.126968,0.135358,0.144223,0.155983,0.156481
Genus,0.096332,0.103203,0.112258,0.117645,0.127490,0.175020
Species,0.088573,0.094791,0.101513,0.107341,0.114751,0.142785


In [0]:
m.load_weights('./PCR_SILVA_all/W2V/ResNet_like_N_big.hdfs')
evaluate_multioutput_on_different_length(model=m, lengths=[25,50,75,100,125,'full_HVR'],data=test,
                                         batch_size=250,save_path='./PCR_SILVA_all/ResNet_like_N_big.csv')

174500/174500 [==============================] - 179s 1ms/step


,25,50,75,100,125,full_HVR
Phylum,0.932785,0.988361,0.995788,0.997215,0.997897,0.998573
Order,0.908894,0.980877,0.993135,0.995983,0.997135,0.997971
Class,0.837788,0.952762,0.980315,0.988275,0.992338,0.994751
Family,0.783599,0.928103,0.968189,0.980774,0.987129,0.990951
Genus,0.639570,0.818957,0.892218,0.925696,0.946195,0.960630
Species,0.482029,0.615461,0.674751,0.706178,0.725656,0.743542


In [0]:
m.load_weights('./PCR_SILVA_all/W2V/ResNet_like_N_big_len50.hdfs')
evaluate_multioutput_on_different_length(model=m, lengths=[25,50,75,100,125,'full_HVR'],data=test,
                                         batch_size=250,save_path='./PCR_SILVA_all/ResNet_like_N_big_len50.csv')

174500/174500 [==============================] - 177s 1ms/step


,25,50,75,100,125,full_HVR
Phylum,0.924046,0.988034,0.995880,0.997335,0.998074,0.998728
Order,0.895072,0.980315,0.993450,0.996309,0.997473,0.998212
Class,0.820407,0.952138,0.981261,0.989181,0.993341,0.995307
Family,0.758321,0.928069,0.970453,0.982676,0.988774,0.992052
Genus,0.608722,0.819691,0.898785,0.931375,0.951135,0.964069
Species,0.462802,0.617289,0.683599,0.715089,0.734223,0.752642


In [0]:
m.load_weights('./PCR_SILVA_all/W2V/Fixed_len_ResNet_like_N.hdfs')
evaluate_multioutput_on_different_length(model=m, lengths=[25,50,75,100,125,'full_HVR'],data=test,
                                         batch_size=250,save_path='./PCR_SILVA_all/Fixed_len_ResNet_like_N.csv')

174500/174500 [==============================] - 182s 1ms/step


,25,50,75,100,125,full_HVR
Phylum,0.818332,0.963444,0.993685,0.997599,0.998195,0.998648
Order,0.752659,0.943438,0.990063,0.996367,0.997370,0.998000
Class,0.639914,0.890361,0.973536,0.990350,0.993920,0.995289
Family,0.546023,0.844063,0.957175,0.984332,0.989662,0.992034
Genus,0.365140,0.699032,0.872774,0.935834,0.952716,0.960464
Species,0.269479,0.520006,0.660269,0.718779,0.735628,0.742556


In [0]:
m.load_weights('./PCR_SILVA_all/W2V/ResNet_like_N_No_W2V.hdfs')
evaluate_multioutput_on_different_length(model=m, lengths=[25,50,75,100,125,'full_HVR'],data=test,
                                         batch_size=250,save_path='./PCR_SILVA_all/ResNet_like_N_No_W2V.csv')

174500/174500 [==============================] - 174s 999us/step


,25,50,75,100,125,full_HVR
Phylum,0.210309,0.210309,0.210309,0.210309,0.210309,0.210309
Order,0.206648,0.206648,0.206648,0.206648,0.206648,0.206648
Class,0.068636,0.068636,0.068636,0.068636,0.068636,0.068636
Family,0.066607,0.066607,0.066607,0.066607,0.066607,0.066607
Genus,0.064905,0.064905,0.064905,0.064905,0.064905,0.064905
Species,0.062287,0.062287,0.062287,0.062287,0.062287,0.062287


### MLP models evaluation

In [0]:
#MLP_no_W2V_no_kmer_len50
test  = pd.read_pickle('PCR_SILVA_all/data/V2_test.pkl')
test  = test.sample(frac=1).reset_index(drop=True)
test  = test.drop(columns=['Complete_species','ambiguity_count'])

m.load_weights('./PCR_SILVA_all/W2V/MLP_no_W2V_no_kmer_len50.hdfs')
evaluate_multioutput_on_different_length(model=m, lengths=[25,50,75,100,125,'full_HVR'],data=test,
                                         batch_size=250,save_path='./PCR_SILVA_all/MLP/MLP_no_W2V_no_kmer_len50.csv')

174500/174500 [==============================] - 18s 102us/step


,25,50,75,100,125,full_HVR
Phylum,0.400178,0.400178,0.400178,0.400178,0.400178,0.400178
Order,0.302023,0.302023,0.302023,0.302023,0.302023,0.302023
Class,0.203181,0.203181,0.203181,0.203181,0.203181,0.203181
Family,0.136946,0.136946,0.136946,0.136946,0.136946,0.136946
Genus,0.135622,0.135622,0.135622,0.135622,0.135622,0.135622
Species,0.122034,0.122034,0.122034,0.122034,0.122034,0.122034


In [0]:
#MLP_no_W2V_len50
m.load_weights('./PCR_SILVA_all/W2V/MLP_no_W2V_len50.hdfs')
evaluate_multioutput_on_different_length(model=m, lengths=[25,50,75,100,125,'full_HVR'],data=test,
                                         batch_size=250,save_path='./PCR_SILVA_all/MLP/MLP_no_W2V_len50.csv')

174500/174500 [==============================] - 18s 102us/step


,25,50,75,100,125,full_HVR
Phylum,0.896963,0.984149,0.994521,0.996705,0.997691,0.998269
Order,0.876080,0.977593,0.992378,0.995748,0.997072,0.997725
Class,0.787954,0.951364,0.980327,0.988544,0.993037,0.994831
Family,0.747484,0.928487,0.969989,0.982499,0.989066,0.991897
Genus,0.606195,0.825330,0.899817,0.931364,0.951794,0.962315
Species,0.464779,0.622900,0.683696,0.715043,0.736206,0.749238


In [0]:
#MLP_no_W2V_len50_fresh
m.load_weights('./PCR_SILVA_all/W2V/MLP_no_W2V_len50_fresh.hdfs')
evaluate_multioutput_on_different_length(model=m, lengths=[25,50,75,100,125,'full_HVR'],data=test,
                                         batch_size=250,save_path='./PCR_SILVA_all/MLP/MLP_no_W2V_len50_fresh.csv')

174500/174500 [==============================] - 18s 103us/step


,25,50,75,100,125,full_HVR
Phylum,0.896888,0.983496,0.993845,0.996321,0.997324,0.997776
Order,0.861782,0.975742,0.991358,0.995266,0.996762,0.997610
Class,0.774223,0.946716,0.978688,0.987284,0.992676,0.994424
Family,0.726321,0.922229,0.966252,0.980688,0.988011,0.991123
Genus,0.587977,0.818126,0.894327,0.929616,0.949925,0.961811
Species,0.452923,0.617014,0.677914,0.709633,0.728138,0.741203


In [0]:
# MLP_W2V_len50
m.load_weights('./PCR_SILVA_all/W2V/MLP_W2V_len50.hdfs')
evaluate_multioutput_on_different_length(model=m, lengths=[25,50,75,100,125,'full_HVR'],data=test,
                                         batch_size=250,save_path='./PCR_SILVA_all/MLP/MLP_W2V_len50.csv')

174500/174500 [==============================] - 17s 98us/step


,25,50,75,100,125,full_HVR
Phylum,0.872481,0.981536,0.993140,0.996143,0.997490,0.998384
Order,0.856745,0.973857,0.990493,0.995100,0.996974,0.997794
Class,0.778241,0.945731,0.977335,0.987077,0.992625,0.994739
Family,0.730670,0.923456,0.967203,0.981186,0.988562,0.991891
Genus,0.606470,0.821341,0.897037,0.929438,0.950246,0.962625
Species,0.460160,0.620017,0.683324,0.712292,0.733209,0.747077
